In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools
  ! pip install ipyfilechooser

In [11]:
# import the required libraries
from czitools.metadata_tools import czi_metadata as czimd
from czitools.utils import misc, planetable
from ipyfilechooser import FileChooser
from IPython.display import display, HTML
import os
import requests
import ipywidgets as widgets
import glob

### Define Parameters for Data Loading

In [4]:
# try to find the folder with data and download otherwise from GitHub.

# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

In [5]:
if not IN_COLAB:
    # choose local file
    fc = FileChooser()
    fc.default_path = INPUT_FOLDER
    fc.filter_pattern = '*.czi'
    display(fc)

elif IN_COLAB:
    # lislt files inside the folder om gdrive
    czifiles = glob.glob(os.path.join(INPUT_FOLDER, "*.czi"))
    wd = widgets.Select(
        options=czifiles,
        description='CZI Files:',
        layout={'width': 'max-content'}
    )
    display(wd)

FileChooser(path='F:\Github\czitools\data', filename='', title='', show_hidden=False, select_desc='Select', ch…

In [6]:
if not IN_COLAB:
    filepath = fc.selected
elif IN_COLAB:
    filepath = wd.value

print(f"Selected File: {filepath}")

Selected File: F:\Github\czitools\data\WP96_4Pos_B4-10_DAPI.czi


In [7]:
# get only specific metadata
czi_dimensions = czimd.CziDimensions(filepath)
print("SizeS: ", czi_dimensions.SizeS)
print("SizeT: ", czi_dimensions.SizeT)
print("SizeZ: ", czi_dimensions.SizeZ)
print("SizeC: ", czi_dimensions.SizeC)
print("SizeY: ", czi_dimensions.SizeY)
print("SizeX: ", czi_dimensions.SizeX)

SizeS:  28
SizeT:  None
SizeZ:  None
SizeC:  1
SizeY:  18764
SizeX:  288764


In [8]:
# and get more info
czi_scaling = czimd.CziScaling(filepath)
czi_channels = czimd.CziChannelInfo(filepath)
czi_bbox = czimd.CziBoundingBox(filepath)
czi_objectives = czimd.CziObjectives(filepath)
czi_detectors = czimd.CziDetector(filepath)
czi_microscope = czimd.CziMicroscope(filepath)
czi_sample = czimd.CziSampleInfo(filepath)

In [9]:
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(filepath)

# get the CZI metadata dictionary directly from filename
mdict = czimd.create_md_dict_red(mdata, sort=False, remove_none=True)

# convert metadata dictionary to a pandas dataframe
mdframe = misc.md2dataframe(mdict)

# create a ipywdiget to show the dataframe with the metadata
wd1 = widgets.Output(layout={"scrollY": "auto", "height": "300px"})

with wd1:
    display(HTML(mdframe.to_html()))
display(widgets.VBox(children=[wd1]))

In [10]:
# write XML to disk
xmlfile = czimd.writexml(filepath)
print("XML File written to:", xmlfile)

XML File written to: F:\Github\czitools\data\WP96_4Pos_B4-10_DAPI_CZI_MetaData.xml


In [13]:
# get the planetable for the CZI file
pt, savepath = planetable.get_planetable(filepath,
                                         norm_time=True,
                                         save_table=True,
                                         planes={"time": 0, "channel": 0, "zplane": 0})

# create a ipywdiget to show the dataframe with the metadata
wd2 = widgets.Output(layout={"scrollY": "auto", "height": "300px"})

with wd2:
    display(HTML(pt.to_html()))
display(widgets.VBox(children=[wd2]))

2025-10-11 17:48:11,945 - czitools - INFO - CZI dimensions found: ['S', 'C']


Reading subblock planes: 0 2Dplanes [00:00, ? 2Dplanes/s]

2025-10-11 17:48:12,004 - czitools - INFO - Planetable saved successfully at: F:\Github\czitools\data\WP96_4Pos_B4-10_DAPI_planetable.csv
